In [1]:
%%html
<script defer src="https://use.fontawesome.com/releases/v5.0.8/js/all.js" integrity="sha384-SlE991lGASHoBfWbelyBPLsUlwY1GwNDJo3jSJO04KZ33K2bwfV9YBauFfnzvynJ" crossorigin="anonymous"></script>
<script disableClusteringAtZoom: 11 to L.markerClusterGroup({})></script>

In [2]:
from ipywidgets import *
from IPython.display import IFrame
from IPython.core.display import display
import subprocess, sys
import webbrowser
from ipywidgets import HTML
import base64
import csv ## https://docs.python.org/fr/3.6/library/csv.html 
import os ## Pour utiliser la methode ci-dessous Get Current Worked Directory pour lire la page Web crée dans le dossier courant.
Lieu = os.getcwd()  ## methode Get Current Worked Directory pour lire la page Web crée dans le dossier courant.
###
import os

import subprocess, sys

from time import gmtime, strftime

from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
#################################################
def Recup_Donnees(titre_csv):
    """ Cette fonction initiale et essentielle permet de construire
        une liste de tuples contenant toutes les informations du 
        tableau par lignes. """
    Donnees=[]  ## liste de tuples pour récupérer toutes les données du fichier csv
    Ligne_Donnees=[]  ## sous liste pour créer la liste des tuples de Donnees
    with open(titre_csv, "r", encoding="utf-8") as texte:  ## Ouverture du csv en mode utf 8 pour encodage des caractères français
        Les_lignes = csv.reader(texte, delimiter=';') ## lecture de chaque ligne du fichier csv
        for ligne in Les_lignes:  ## bouclage sur chaque ligne des lignes
            for element in ligne:  ## Récupération de chaque élément de chaque ligne
                Ligne_Donnees.append(element)  ## création d'1 tuple avec tous les éléments d'1 même ligne
            Ligne_Donnees=tuple(Ligne_Donnees)  ## assurance qu'il s'agit bien d'un tuple ..?
            Donnees.append(Ligne_Donnees)  ## Ajout du tuple créé à la liste de tuple Données
            Ligne_Donnees=[]  ## Réinitialisation d'1 futur 'tuple' vide pour boucler sur la ligne suivant du tableau
    longueur_Donnees=len(Donnees)  ## Détermination de la longueurs des Données
    return Donnees,longueur_Donnees
Donnees,longueur_Donnees=Recup_Donnees("Entreprises_Complet.csv")
# print("longueur = ",longueur_Donnees)
Nb_entreprise=1

for ligne in range(0,longueur_Donnees):
    tupl=Donnees[ligne]
    li=list(tupl)
#     print(" ***** Ligne = ",ligne," ***********")
#     print()
#     print(li)
    result_Nom_entreprise=li[37]
    if result_Nom_entreprise != "":
        Nb_entreprise += 1
# print("Nb_entreprise = ",Nb_entreprise)

In [3]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster,LayersControl,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    Popup,
    GeoJSON,
    DrawControl,
    basemaps, FullScreenControl,
    SearchControl, AwesomeIcon, Icon, ScaleControl
)

from ipywidgets import HTML

In [4]:
def select_entreprises_fonc(change):
    global selection
    sel=change['new']
    sel=list(sel)
    ty=type(sel)
    print("type =",ty)
    print("selection = ",sel)
    selection = sel
    return selection

In [5]:
List_Nom_Entrprise = []
for ligne in range(2,Nb_entreprise+1):
    tupl=Donnees[ligne]
    li=list(tupl)
    result_Nom=li[2]
    List_Nom_Entrprise.append(result_Nom)
List_Nom_Entrprise.sort()

select_entreprises = widgets.SelectMultiple(
    options=List_Nom_Entrprise ,
    #value=['Oranges'],
    rows=5,

    disabled=False,
    layout={'width': 'max-content'}
)
select_entreprises.observe(select_entreprises_fonc,names='value')
# display(select_entreprises)
#     description='Entreprises',

In [6]:
Selecti = VBox([widgets.Label(value='1) Choisissez, ci-dessous, vos entreprises (Ctrl pour plusieurs).'),select_entreprises])

In [7]:
def Create_pdf(_):
    global selection
    print("apres btn pdf, selection =",selection)
    infos = []
    Long_selection = len(selection)
    for select in range (Long_selection):
        for ligne in range(1,Nb_entreprise+1): # 19):
            tupl=Donnees[ligne]
            li=list(tupl)
        #     print(" ***** Ligne = ",ligne," ***********")
        #     print()
        #     print(li)
            result=li[2]
            if selection[select] == result:
                
                infos.append(str(li[2]))
                infos.append(str(li[0]))
                infos.append(str(li[1]))
                infos.append(str(li[32]))
                infos.append(str(li[33]))
                infos.append(str(li[34]))
                infos.append("--> Tel : "+str(li[35]))
                infos.append("--> Mail : "+str(li[43]))
                infos.append("--> Lien : "+str(li[36]))
                infos.append("**")

                
    ###
    doc = SimpleDocTemplate("Infos_Lieux_Stages.pdf",pagesize=letter,
                            rightMargin=72,leftMargin=72,
                            topMargin=72,bottomMargin=18)
    Story=[]
    logo = "Lib2.png"
#     magName = "Pythonista"
#     issueNum = 12

    formatted_time =strftime("%Y-%m-%d %H:%M:%S") # strftime('%A %d/%m/%Y %H:%M:%S') # strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()) # time.ctime()

    address_parts = infos
    im = Image(logo, 2*inch, 2*inch)
    Story.append(im)
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    ptext = '%s' % formatted_time
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 12))
     
    for part in address_parts:
        ptext = '%s' % part.strip()
        if ptext == "**":
            Story.append(Spacer(1, 12))
        else:
            Story.append(Paragraph(ptext, styles["Normal"]))   
    Story.append(Spacer(1, 12))

    doc.build(Story)


In [8]:
# yep=widgets.HTML(
#     value="<a href='../../files/Infos_Lieux_Stages.pdf' target='_blank'>*</a>")
# # yep

In [9]:
yep=widgets.HTML(
    value="<a href='../../files/Infos_Lieux_Stages.pdf' target='_blank'><i class='fa fa-play' aria-hidden='true' color='#33CC66'></i></a>")
# 33CC66 # 66FF99

In [10]:
# <a href="http://www.website.com" title="Website name"><i style="margin-right: 0.5em; color: #EEEEEE;" class="icon-home icon-4x"></i>Website Link</a>

In [11]:
#######################################
btn_pdf = Button(icon="file",button_style='danger') # cogs  # angle-left # file-pdf-o # spinner
btn_pdf.layout.width="30px"
btn_pdf.on_click( Create_pdf)
# display(btn_pdf)
#################

In [12]:
ligne=1
tupl=Donnees[ligne]
li=list(tupl)
Nom_lycee=li[2]
#################################
Nb="L'établissement '"+str(Nom_lycee)+"' propose "+str(Nb_entreprise-2)+" adresses de lieux de stages :"
Nb2 = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p><i>'+ Nb +' </i></p>')
Nb2.layout.max_width='200px'
Nb2.layout.border = '1px solid black'

In [13]:
#################################
Tex2="Créez votre PDF d'informations des lieux de stages :"
Texto2 = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p><u><b>'+ Tex2 +' </b></u></p>')
# Tex1a = "1) Choisissez vos entreprises (Ctrl pour plusieurs)."
# Texto1a = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Tex1a +' </p>')

# Tex1b = "2) Cliquez sur le bouton 'Creez le PDF'."
# Texto1b = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Tex1b +' </p>')

# Tex1c = "3) Cliquez sur le bouton 'Ouvrir de PDF'."
# Texto1c = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Tex1c +' </p>')


Tex3="2) Créer le PDF --> "
Texto3 = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Tex3 +' </p>')
Btn_CrPDF = HBox([Texto3,btn_pdf])

Tex4="3) Ouvrir le PDF --> "
Texto4 = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Tex4 +' </p>')
Btn_OuvrPDF = HBox([Texto4,yep])

Btn = VBox([Btn_CrPDF,Btn_OuvrPDF])
Boite = HBox([Selecti, Btn])

Texte_w = VBox([Texto2,Boite]) # Texto1a,Texto1b,Texto1c, 

Texte_w.layout.max_width='640px'
Texte_w.layout.border = '1px solid black'
#################################
# Texte_w
####################################
# TexDrop1="Choisissez les domaines d'entreprises"
# TexDrop1a = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p><u><b>'+ TexDrop1 +' </b></u></p>')
# TexDrop2="visibles sur la carte :"
# TexDrop2a = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p><u><b>'+ TexDrop2 +' </b></u></p>')
# dropi=VBox([TexDrop1a,TexDrop2a,drop])
# dropi.layout.max_width='340px'
# dropi.layout.border = '1px solid black'

In [14]:
file = open("Lib2.png", "rb")
image = file.read()
im_lycee = widgets.Image(
    value=image,
    format='png',
    width=200,
    height=200,
)


In [15]:
file = open("titre2.png", "rb")
image = file.read()
im_titre = widgets.Image(
    value=image,
    format='png',
    width=400,
    height=50,
)
im_titre
Box_im_titre=Box([im_titre])
Box_im_titre.layout.border = '1px solid black'
Box_im_titre

Box(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02`\x00\x00\x00\x95\x08\x02\x00\x00\x…

In [16]:
Menus = HBox([im_lycee,Nb2,Texte_w], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))
display(Menus)

type = <class 'list'>
selection =  ['ANSES (BCM-Alimentaire)']
type = <class 'list'>
selection =  ['ANSES (BCM-Alimentaire)', 'ANTARTIC ']
apres btn pdf, selection = ['ANSES (BCM-Alimentaire)', 'ANTARTIC ']
type = <class 'list'>
selection =  ['A.R.D.']
apres btn pdf, selection = ['A.R.D.']


In [17]:
from ipyleaflet import (
    Map, basemaps, basemap_to_tiles,
    Circle, Marker, Rectangle, LayerGroup, LayersControl,
    AwesomeIcon, Popup, ScaleControl, FullScreenControl,
    SearchControl
)
from ipywidgets import HTML
###################################
names = []
##################################

cent = [49.25112, 4.0271] #drop.value#
zoom = 20
m = Map(center=cent, zoom=zoom)
m.add_control(FullScreenControl())
marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

m.add_control(SearchControl(
  position="topleft",
  url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
  zoom=15,
  marker=marker
))


m.interact(zoom=(5,20,1))
###############################################################
""" Création de la liste (names) des différents domaines d'entreprise"""
for ligne in range(1,Nb_entreprise+1):
    faire = 1
    tupl=Donnees[ligne]
    li=list(tupl)
#     print(" ***** Ligne = ",ligne," ***********")
#     print()
#     print(li)
    result=li[37]
    if result !="":
        long_names=len(names)
        for i in range(long_names):
            if names[i]==li[0]:
                faire=0
        if faire==1:
            names.append(li[0])
###########################################################
"""Création des noms de liste de domaine nommées markers_domain"""
noms_domaine = []
long_names=len(names)
for name in range(long_names):
    nom="markers_"+names[name]
    noms_domaine.append(nom)
# print(noms_domaine)

dico_noms_domaine = {n: [] for n in noms_domaine}
#############################################################
"""Création des noms de liste de couches nommées couches_domain"""
couches_domain = []
long_names=len(names)
for name in range(long_names):
    nom="couches_"+names[name]
    couches_domain.append(nom)
# print(noms_domaine)

list_couches_domain = [n for n in couches_domain]
# print(list_couches_domain)
#############################################################
for ligne in range(1,Nb_entreprise+1): # longueur_Donnees
    tupl=Donnees[ligne]
    li=list(tupl)
#     print(" ***** Ligne = ",ligne," ***********")
#     print()
#     print(li)
    result=li[37]
    if result !="":
    #     print(result)
    #     print(type(result))
        result=result.split(",")
        latitude=result[0]
        longitude=result[1]
        Coord = [latitude, longitude]
    #     print("latitude=",latitude)

        infos="<u>Entreprise :</u> <b>"+str(li[2])+"</b> : <br />"+str(li[0])+" : "+str(li[1])+". <br />"+str(li[32])+", "+str(li[33])+" "+str(li[34])+"<br/> --> Tel : "+str(li[35])+"<br/> --> Mail : "+str(li[43])+"<br/> --> <a target='_blank' rel='noopener noreferrer' href='"+str(li[36])+"'>Lien </a>"
    ##### https://www.flaticon.com/fr/icone-premium/laboratoire-medical_3483814?related_id=3483663&origin=search    
        if li[0] == "Lycée":
            icon = AwesomeIcon(name='fa-graduation-cap', marker_color='lightred', icon_color='black', icon_size=[40, 40])
    #         icon = Icon(icon_url='https://img-premium.flaticon.com/png/512/3483/3483835.png?token=exp=1627657167~hmac=914e0b35ab63604b16469288e1da5c19', icon_size=[30, 30], icon_anchor=[10,10])
      # fa-graduation-cap
    # bcm --> cut ; genetique --> random ; fermentation --> hourglass-start
        if li[0] == "Chimie":
            icon = AwesomeIcon(name='flask', marker_color='beige', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Médical":
            icon = AwesomeIcon(name='heartbeat', marker_color='red', icon_color='black' , icon_size=[40, 40])
             
        if li[0] == "BCM":
            icon = AwesomeIcon(name='cut', marker_color='lightgray', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Génétique":
            icon = AwesomeIcon(name='random', marker_color='lightgray', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Fermentations":
            icon = AwesomeIcon(name='hourglass-start', marker_color='lightgray', icon_color='black' , icon_size=[40, 40])

        if li[0] == "Microbiologie":
            icon = AwesomeIcon(name='link', marker_color='lightgray', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Environnement":
            icon = AwesomeIcon(name='leaf', marker_color='lightgreen', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Pharmaceutique":
            icon = AwesomeIcon(name='pills', marker_color='lightblue', icon_color='black' , icon_size=[40, 40])
    #         icon = Icon(icon_url='https://img-premium.flaticon.com/png/512/3483/3483663.png?token=exp=1627657677~hmac=01ec1a6d6f237b1bdea0e5db5a905f46', icon_size=[30, 30], icon_anchor=[10,10])
        if li[0] == "Cosmétique":
            icon = AwesomeIcon(name='eye', marker_color='pink', icon_color='black' , icon_size=[40, 40])
    #         icon = Icon(icon_url='https://img-premium.flaticon.com/png/512/3483/3483814.png?token=exp=1627657700~hmac=a4c57b57f1129517d16ef9af1b5306ca', icon_size=[30, 30], icon_anchor=[10,10])
        if li[0] == "Alimentaire":
            icon = AwesomeIcon(name='coffee', marker_color='orange', icon_color='black', icon_size=[40, 40])
    #         icon = Icon(icon_url='https://img-premium.flaticon.com/png/512/3483/3483835.png?token=exp=1627657167~hmac=914e0b35ab63604b16469288e1da5c19', icon_size=[30, 30], icon_anchor=[10,10])

        mark = Marker(location=Coord, draggable=False, icon=icon, name=li[0]) # markerColor: 'green' # marker_color

        #########
        """Remplis les listes 'markers_domaine des différents points de marqueurs'"""
#         yo=0
#         print("yo=",yo)
#         print("li[0]=",li[0])
#         print("long_names=",long_names)
#         print("name=",name)
        for name in range(long_names):
            if li[0] == names[name]:
#                 print("ok, on a : li[0] == names[name]")
                listo = dico_noms_domaine[noms_domaine[name]]
#                 print("voici listo avant :",listo)
                listo.append(mark) 
#                 print("voici listo après :",listo)
#                 print("***")
#             print("*** ****")

        #########        

        ##########################################

        message2 = HTML()
        message2.value = infos
        # Popup with a given location on the map:
        popup = Popup(
            location=Coord,
            close_button=False,
            auto_close=False,
            close_on_escape_key=False
        ) #        child=message2,

        mark.popup = message2

m.add_control(ScaleControl(position='bottomleft'))
# m
#####################################################""
# Create some layers



    #########
"""Remplis les listes 'markers_domaine des différents points de marqueurs'"""
for name in range(long_names):
#     if li[0] == names[name]:
#     print("**")
#     print("name =",name)
    listo = dico_noms_domaine[noms_domaine[name]]
#     print("listo =",listo)
#     a=list_couches_domain[name]
#     print("list_couches_domain[name]",a)
#     b=str(names[name])
#     print("nom de la couche : ",b)
#     print("########################")
    list_couches_domain[name] = LayerGroup(name =str(names[name]), layers=listo) 
    m.add_layer(list_couches_domain[name])


    #########       

# layer_group1 = LayerGroup(name = "couche1", layers=markers_lycee) 
# layer_group2 = LayerGroup(name = "couche2", layers=markers_pharma)
# layer_group3 = LayerGroup(name = "couche3", layers=markers_alim)
# layer_group4 = LayerGroup(name = "couche4", layers=markers_cosmeto)
# m.add_layer(layer_group1)
# m.add_layer(layer_group2)
# m.add_layer(layer_group3)
# m.add_layer(layer_group4)

control = LayersControl(position='topleft')
m.add_control(control)
# print(names)
m

Map(center=[49.25112, 4.0271], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…